# Transfer Learning for tabular Data

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs

### Data Preparation

In [2]:
x,y = make_blobs(n_samples=8000,n_features=2,random_state=5,centers=3,cluster_std=2)
x.shape,y.shape

((8000, 2), (8000,))

In [3]:
pd.DataFrame(x)

,0,1
0,-5.155103,11.671184
1,-7.474192,7.641312
2,-4.919851,7.465616
3,-5.795091,6.014681
4,-2.433272,10.665887
...,...,...
7995,-7.260874,11.656437
7996,-0.484286,3.444093
7997,-3.492756,7.648294
7998,-7.706737,7.379782


In [4]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
y.shape

(8000, 3)

In [5]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(x,y,test_size=0.2,random_state=5)
print(xtr.shape,xts.shape,ytr.shape,yts.shape)

(6400, 2) (1600, 2) (6400, 3) (1600, 3)


## Modelling the network

In [7]:
from tensorflow.keras import models,layers

In [ ]:
ip = layers.Input(shape=(2,),name='model_input')
d1 = layers.Dense(10,activation='relu',name='dense_1')(ip)
d2 = layers.Dense(8,activation='relu',name='dense_2')(d1)

op = layers.Dense(3,activation='softmax',name='model_output')(d2)

model = models.Model(inputs=ip,outputs=op,name='3class_classifier')
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(xtr,ytr,epochs=100,validation_data=(xts,yts))

In [ ]:
model.fit(xtr,ytr,epochs=100,validation_data=(xts,yts))

In [10]:
model.save("model.h5",save_format='tf')

# Transfer Learning

Refrence - https://www.tensorflow.org/guide/keras/save_and_serialize#hdf5_format

In [27]:
x,y = make_blobs(n_samples=500,n_features=2,random_state=5,centers=5,cluster_std=2)
x.shape,y.shape

((500, 2), (500,))

In [ ]:
pd.DataFrame(x)

In [28]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
y.shape

(500, 5)

In [14]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(x,y,test_size=0.2,random_state=5)
print(xtr.shape,xts.shape,ytr.shape,yts.shape)

(400, 2) (100, 2) (400, 5) (100, 5)


In [15]:
#loading the pretrained model
premodel = models.load_model("model.h5")
premodel.summary()

Model: "3class_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_input (InputLayer)     [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
model_output (Dense)         (None, 3)                 27        
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________


In [16]:
premodel.layers

In [17]:
premodel.summary()

Model: "3class_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_input (InputLayer)     [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
model_output (Dense)         (None, 3)                 27        
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________


#### Drop the Last layer of the model - which has 3 neurons and softmax activation

In [29]:
newmodel = models.Model(premodel.input,premodel.get_layer("dense_2").output)
newmodel.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_input (InputLayer)     [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
Total params: 118
Trainable params: 0
Non-trainable params: 118
_________________________________________________________________


### Approach 1

#

In [19]:
ip = layers.Input(shape=(2,))
last_layer = newmodel(ip)
last_layer

<KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'model')>

In [20]:
op = layers.Dense(5,activation='softmax')(last_layer)

In [21]:
final_model = models.Model(ip,op)
final_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
model (Functional)           (None, 8)                 118       
_________________________________________________________________
dense (Dense)                (None, 5)                 45        
Total params: 163
Trainable params: 163
Non-trainable params: 0
_________________________________________________________________


### Approach 2

In [30]:
premodel = models.load_model("model.h5")
# remove the last layer of the pretrained network
newmodel = models.Model(premodel.input,premodel.get_layer("dense_2").output)

In [31]:
cl = newmodel.layers[-1].output
op = layers.Dense(5,activation='softmax',name='new_output')(cl)
final_model = models.Model(newmodel.input,op)
final_model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_input (InputLayer)     [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
new_output (Dense)           (None, 5)                 45        
Total params: 163
Trainable params: 163
Non-trainable params: 0
_________________________________________________________________


In [24]:
for layer in final_model.layers[:-1]:
  layer.trainable=False
final_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_input (InputLayer)     [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
new_output (Dense)           (None, 5)                 45        
Total params: 163
Trainable params: 45
Non-trainable params: 118
_________________________________________________________________


In [25]:
final_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
final_model.fit(xtr,ytr,epochs=50,validation_data=(xts,yts))

Epoch 1/50
13/13 [==============================] - 0s 16ms/step - loss: 12.1074 - accuracy: 0.2000 - val_loss: 11.4658 - val_accuracy: 0.2000
Epoch 2/50
13/13 [==============================] - 0s 3ms/step - loss: 11.3170 - accuracy: 0.2000 - val_loss: 10.6732 - val_accuracy: 0.2000
Epoch 3/50
13/13 [==============================] - 0s 3ms/step - loss: 10.5397 - accuracy: 0.2025 - val_loss: 9.8789 - val_accuracy: 0.2000
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 9.7690 - accuracy: 0.2050 - val_loss: 9.1238 - val_accuracy: 0.2000
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 9.0298 - accuracy: 0.2200 - val_loss: 8.3973 - val_accuracy: 0.2300
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 8.3653 - accuracy: 0.2500 - val_loss: 7.7025 - val_accuracy: 0.2800
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 7.7280 - accuracy: 0.2575 - val_loss: 7.1060 - val_accuracy: 0.3200
Epoch 8/50
13/